In [1]:
import clean_database
import DataAugmentation
import Feature_Engineering as fe
import train_test_validation as ttv
import graficar
import linear
from linear import Linear_Model
import logistic
import mixture

#from mlp import MultilayerPerceptron
import numpy as np
import jax.numpy as jnp
import y_hot as hot
import mlp_mlflow
from mlp_mlflow import MultilayerPerceptron
import mlflow
import importlib

Limpieza de la base de datos
=====================================

In [2]:
datos_bajos,datos,header=clean_database.Clean_Data()
print(header)

3364 6239


c:\Users\52333\Documents\doctorado\ml-md\proyecto\clean_database.py:379: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfd=d1.append(d2)


['clase', 'P1_1', 'P1_2_1', 'P1_2_2', 'P1_3', 'P1_4_1', 'P1_4_2', 'P1_4_3', 'P1_4_4', 'P1_4_5', 'P1_4_6', 'P4_1_1', 'P4_1_2', 'P4_1_3', 'ENT_x', 'SEXO', 'EDAD', 'P3_1', 'PA3_1', 'PA3_2', 'PA3_3_NIVEL', 'PA3_3_BIMESTRE', 'PA3_3_TRIMESTRE', 'PA3_3_CUATRIMESTRE', 'PA3_3_SEMESTRE', 'PA3_3_ANIO', 'PA3_5', 'PA3_7_1', 'PA3_8_1', 'PA3_8_2', 'PA3_8_3', 'PA3_8_4', 'PA3_8_5', 'PA3_8_6', 'PA3_8_7', 'PA3_8_8', 'PB3_1', 'PB3_3', 'FILTRO_A', 'PB3_4', 'PB3_5_NIVEL', 'PB3_5_BIMESTRE', 'PB3_5_TRIMESTRE', 'PB3_5_CUATRIMESTRE', 'PB3_5_SEMESTRE', 'PB3_5_ANIO', 'PB3_6', 'PB3_9_1', 'PB3_9_2', 'PB3_9_3', 'PB3_10_1', 'PB3_10_2', 'PB3_10_3', 'PB3_10_4', 'PB3_10_5', 'PB3_11_1', 'PB3_11_2', 'PB3_11_3', 'PB3_11_4', 'PB3_11_5', 'PB3_12_1', 'PB3_12_2', 'PB3_12_3', 'PB3_12_4', 'PB3_12_5', 'PB3_12_6', 'PB3_12_7', 'PB3_12_8', 'FILTRO_B', 'PB3_13_1', 'PB3_13_2', 'PB3_13_3', 'PB3_13_4', 'PB3_13_5', 'PB3_13_6', 'PB3_13_7', 'PB3_14', 'PB3_15', 'PB3_16_1', 'PB3_16_2', 'PB3_16_3', 'PB3_16_4', 'PB3_16_5', 'FILTRO_C', 'PC3_1',

In [3]:
print(header[63])

PB3_12_4


Data augmentation para la clase de datos '1' (alumnos con bajo desempeño escolar)
=====================================================================================

In [4]:
datos_aumentados=DataAugmentation.Smote(datos_bajos)
print(datos_aumentados.shape)
datos=np.concatenate((datos,datos_aumentados),axis=0)
#datos=datos[np.random.permutation(datos.shape[0]),:]
x,y=datos[:,1:],datos[:,0]



(3364, 96)


Feature Engineering (Backwards selection)
===============================================

In [5]:
importlib.reload(fe)
x1,x2=fe.separate(x,y) #tenemos que separar los features en
size=x1.shape[1]
min_group=27 #son los features con los que nos queremos quedar
x1,x2,ganadores=fe.sequential_backwards(size,x1,x2,min_group)
x=x[:,ganadores]
x=x.round(1)
x_train, x_test, x_v,y_train, y_test, y_v=ttv.split_data(x,y)

np.savetxt('datosx.csv',x_train,delimiter=',')
np.savetxt('datosy.csv',y_train,delimiter=',')

Los índices ganadores fueron: (26, 29, 30, 32, 33, 34, 37, 45, 47, 48, 53, 54, 56, 57, 58, 60, 61, 62, 63, 65, 66, 77, 78, 79, 80, 83)


In [36]:
import ada_boosting as ab
import decision_categorical
importlib.reload(ab)
importlib.reload(decision_categorical)
importlib.reload(linear)
num_stumps=100


modelo='lineal'
run_name='Ada_lineal'
num_iter=10
ab.train_boosting_lineal(x_train, y_train,x_v,y_v,modelo, num_stumps,run_name,num_iter)
#y_hat=ab.Ada_Boost(x_train,y_train,x_v,y_v,M=10)

Vamos en el experimento 0
EL Y_HAT SIN RAVEL [[ 1.1828    ]
 [ 0.47744292]
 [ 0.47744292]
 ...
 [-0.5747176 ]
 [-1.5532742 ]
 [ 0.47744292]]
EL Y_HAT CON sign [[ 1.]
 [ 1.]
 [ 1.]
 ...
 [-1.]
 [-1.]
 [ 1.]]
EL Y_HAT CON RAVEL [ 1  1  1 ... -1 -1  1]
(array([-1,  1]), array([4816, 3828], dtype=int64))
el yhat100 6507
el yhat100 [ 1  1  1 ... -1 -1  1] [ 1  1  1 ... -1  1  1]
el shape de data (8644,)
Vamos en el experimento 1
EL Y_HAT SIN RAVEL [[0.02921858]
 [0.02921856]
 [0.02921856]
 ...
 [0.02921855]
 [0.02868428]
 [0.02921856]]
EL Y_HAT CON sign [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
EL Y_HAT CON RAVEL [1 1 1 ... 1 1 1]
(array([1]), array([8644], dtype=int64))
el yhat100 4475
el yhat100 [1 1 1 ... 1 1 1] [ 1  1  1 ... -1  1  1]
el shape de data (8644,)
Vamos en el experimento 2
EL Y_HAT SIN RAVEL [[0.0292111]
 [0.0292111]
 [0.0292111]
 ...
 [0.0292111]
 [0.0292111]
 [0.0292111]]
EL Y_HAT CON sign [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
EL Y_HAT CON RAVEL [1 1 1 ... 1 1 1]
(arra

([array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.33827413e-08, 1.33827413e-08, 1.33827413e-08],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [1.33827413e-08, 6.69137067e-09, 1.33827413e-08, ...,
          6.69137067e-09, 1.33827413e-08, 6.69137067e-09],
         [4.67904969e-04, 2.80742981e-04, 4.67904969e-04, ...,
          2.80742981e-04, 4.67904969e-04, 4.67904969e-04],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
  array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.79107507e-16, 1.79107507e-16, 1.79107507e-16],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        

In [18]:
import ada_boosting as ab
import decision_categorical
importlib.reload(ab)
importlib.reload(decision_categorical)
importlib.reload(mlp_mlflow)
num_stumps=100


modelo='mlp'
run_name='Ada_mlp'
num_iter=20
ab.train_boosting_mlp2(x_train, y_train,x_v,y_v,modelo, num_stumps,run_name,num_iter)
#y_hat=ab.Ada_Boost(x_train,y_train,x_v,y_v,M=10)

End of an mlp
Vamos en el experimento 0
End of an mlp
End of an mlp
Vamos en el experimento 1
End of an mlp
End of an mlp
Vamos en el experimento 2
End of an mlp
End of an mlp
Vamos en el experimento 3
End of an mlp
End of an mlp
Vamos en el experimento 4
End of an mlp
End of an mlp
Vamos en el experimento 5
End of an mlp
End of an mlp
Vamos en el experimento 6
End of an mlp
End of an mlp
Vamos en el experimento 7
End of an mlp
End of an mlp
Vamos en el experimento 8
End of an mlp
End of an mlp
Vamos en el experimento 9
End of an mlp
End of an mlp
Vamos en el experimento 10
End of an mlp
End of an mlp
Vamos en el experimento 11
End of an mlp
End of an mlp
Vamos en el experimento 12
End of an mlp
End of an mlp
Vamos en el experimento 13
End of an mlp
End of an mlp
Vamos en el experimento 14
End of an mlp
End of an mlp
Vamos en el experimento 15
End of an mlp
End of an mlp
Vamos en el experimento 16
End of an mlp
End of an mlp
Vamos en el experimento 17
End of an mlp
End of an mlp
Vamos 

([array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          6.93895632e-04, 6.93895632e-04, 6.93895632e-04],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [1.33824908e-08, 6.69124540e-09, 1.33824908e-08, ...,
          6.69124540e-09, 1.33824908e-08, 6.69124540e-09],
         [6.93895632e-04, 6.93895632e-04, 6.93895632e-04, ...,
          4.85726943e-04, 6.93895632e-04, 6.93895632e-04],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
  array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.55051825e-07, 1.55051825e-07, 1.55051825e-07],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        

In [53]:
importlib.reload(ab)
prediction=ab.prediction(x,modelo='lineal')
print(np.unique(prediction,return_counts=True))

0    [1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1,...
1                                           Prediction
2    [1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1,...
Name: Prediction, dtype: object


SyntaxError: '[' was never closed (<unknown>, line 1)

Decision Tree
=================

In [58]:

np.unique(y,return_counts=True)

(array([0., 1.]), array([6239, 6728], dtype=int64))

In [48]:
import decision_categorical
import os
max_depth=5
min_samples=30

gini_min=0.1
file = 'arbol.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
importlib.reload(decision_categorical)
decision_categorical.train_tree(x, y, gini_min,max_depth,current_depth=0,side='None')

KeyboardInterrupt: 

In [61]:
importlib.reload(decision_categorical)
y_hat=decision_categorical.prediccion(x)

c:\Users\52333\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [62]:
print(np.unique(y_hat))

[0]


Multilayer Perceptron
======================================


In [11]:
importlib.reload(mlp_mlflow)
x_trainp,y_trainp=jnp.transpose(x_train),jnp.transpose(y_train)
layers=[x_trainp.shape[0],30,30,2]
lr=0.01
labels=jnp.array([0,1])
k_clases=2
samples=x.shape[0]
y_hotp=jnp.transpose(hot.one_hot(y_train,2))
stop=0.00001
max_steps=10000
x_valp,y_valp,y_hot_valp,samples_val=jnp.transpose(x_v),jnp.transpose(y_v),jnp.transpose(hot.one_hot(y_v,2)),x_v.shape[0]
run_name='multilayer'
mlp=MultilayerPerceptron(layers,lr,labels,k_clases,samples,x_trainp,y_trainp,y_hotp,stop,max_steps)
loss,recall_list,precision_list,loss_list,precision_list_val,recall_list_val,loss_list_val=mlp.modelo(mlp.weights,max_steps,x_trainp,y_trainp,y_hotp,lr,k_clases,samples,labels,stop,x_valp,y_valp,y_hot_valp,samples_val,run_name)


TRAINING: In the epoch     0 the loss is 0.91279 and the accuracy is 0.59474
VALIDATION: In the epoch     0 the loss is 0.84738 and the accuracy is 0.60302
TRAINING:In the epoch     0 the precission is 0.69581 and the recall is 0.72665
VALIDATION:In the epoch     0 the precission is 0.70528 and the recall is 0.72727
TRAINING: In the epoch    10 the loss is 0.71405 and the accuracy is 0.63069
VALIDATION: In the epoch    10 the loss is 0.68239 and the accuracy is 0.62586
TRAINING: In the epoch    20 the loss is 0.62787 and the accuracy is 0.65529
VALIDATION: In the epoch    20 the loss is 0.60924 and the accuracy is 0.65966
TRAINING: In the epoch    30 the loss is 0.57378 and the accuracy is 0.67380
VALIDATION: In the epoch    30 the loss is 0.56115 and the accuracy is 0.67519
TRAINING: In the epoch    40 the loss is 0.53422 and the accuracy is 0.68111
VALIDATION: In the epoch    40 the loss is 0.52461 and the accuracy is 0.68068
TRAINING: In the epoch    50 the loss is 0.50374 and the a

Linear Model
=================

In [9]:

dim=int(x_train.shape[1])
y_trainl=y_train
y_vl=y_v

for i in range(y_train.shape[0]):
    if y_train[i]==0:
        y_trainl[i]==-1
for i in range(y_v.shape[0]):
    if y_v[i]==0:
        y_vl[i]=-1

x_trainl=jnp.array(x_train)
y_trainl=jnp.array(y_trainl)
x_vl=jnp.array(x_v)
y_vl=jnp.array(y_vl)
labels=jnp.array([0,1])
k_classes=2
n_steps=50000
lr=1e-5
samples=x_trainl.shape[0]
#linear model
importlib.reload(linear)
modelo=Linear_Model(dim)
theta = modelo.generate_theta() #inicializamos theta con valores aleatorios
print(x_trainl.shape, y_trainl.shape)
y_trainl=jnp.reshape(y_trainl,(y_train.shape[0],1))
print(x_trainl.shape, y_trainl.shape,dim)
#theta = modelo.gradient_descent(theta, x_trainl, y_trainl, 10, lr = 0.01)
#y_hat=modelo.estimate_grsl(x_trainl,theta)





(8644, 25) (8644,)
(8644, 25) (8644, 1) 25


In [38]:
run_name='lineal'
y_hatl,y_hat_vall=modelo.model(theta, x_trainl, y_trainl, lr,n_steps,x_vl,y_vl,run_name)

este es lr
este es theta [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
este es y_hat de training (8644, 1)
este es y_hat de val (1441, 1)
precision y recall 0.0 0


In [15]:
#Ridge
l=0.5
dim=int(x_train.shape[1])
k_classes=2
importlib.reload(linear)
modelo=Linear_Model(dim)
incremento=0.1
run_name='Ridge'
samples=1441
x_train_r=x_train[:1441]
y_train_r=y_train[:1441]
x_vr=x_v
y_vr=y_v
y_hatR,y_hatR_v,precision_list=modelo.model_ridge(x_train_r,y_train_r,l,incremento,samples,k_classes,labels,x_vr,y_vr,run_name)



Logistic
================

In [18]:
importlib.reload(logistic)
from logistic import Logistic_Regression
x_train_log,y_train_log=jnp.transpose(x_train),jnp.transpose(y_train)
xlogv,ylogv=jnp.transpose(x_v),jnp.transpose(y_v)
model = Logistic_Regression()
alpha=1e-1
tol=1e-3

lbda=0.1
model.fit(x_train_log, y_train_log,alpha,tol,lbda)
Y_hat = model.estimate(x_train_log)
y_hat_v=model.estimate(xlogv)
run_name='logistic'
precision=model.precision(y_train_log, Y_hat,ylogv,y_hat_v,alpha,lbda,run_name)
print(precision)

0.1
lambda 0.1
-11372.53515625
-7184.78271484375
-5446.65185546875
-4493.23486328125
EL ACCURACY de val ES 0.8147119879722595
0.8273947238922119


Mixture of Gaussians
===========================

In [21]:
importlib.reload(mixture)

clases=int(2)


x11,x12=fe.separate(x_train,y_train)
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(features)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=0.4*np.ones(features)
mu2=0.3*np.ones(features)
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_train.shape[0]
print('estps son los mu1 y mu2',mu1,mu2)
ep=0.001
run_name='Mixture'
y_hat,precision=mixture.get_prediction(x_train,mu1,sigma1,mu2,sigma2,features,samples,y_train,ep,run_name)
precision


(13471, 25) (4144, 25)
25
1
esta es la gausiana [1.82049781e-12 1.41125156e-11 3.94850000e-12 1.41125156e-11
 2.07296534e-12 1.41125156e-11 1.41125156e-11 2.34868416e-12
 3.46761012e-12 2.43225856e-12] [1.07749021e-12 3.20126342e-12 4.49557545e-13 7.18951813e-13
 5.62864594e-13 3.59059690e-13 1.41109321e-12 3.31517557e-12
 5.62864594e-13 3.59059690e-13]
este es el denominador [8.35220778e-13 2.79109642e-11 1.98939512e-12 2.79109642e-11
 9.64469192e-13 2.79109642e-11 2.79109642e-11 1.08889204e-12
 1.66725361e-12 1.13163419e-12]
estos son los nk 4023.0095401489466 1562.6943746898914
[0.48481226 0.55661332 0.58363747 0.60877057 0.65487889 0.65490484
 0.61909811 0.43403518 0.45322948 0.46100908 0.57890691 0.47837694
 0.48897113 0.52522824 0.62411234 0.47432035 0.58040977 0.60783645
 0.61642246 0.62265487 0.62316996 0.48859983 0.58338762 0.51544789
 0.60592337] [[0.19910425 0.14114608 0.14785832 0.15505865 0.16669325 0.1668161
  0.18100314 0.15392919 0.14280606 0.14450701 0.1707041  0.13862

0.4757748246192932

In [56]:
clases=int(2)

importlib.reload(mixture)
x11=X[:27940]
x12=X[27940:]
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(x11)
print('este es samples1',samples1)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=0.3*np.ones(features)
mu2=0.4*np.ones(features)
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_v.shape[0]
y_hat,precision=mixture.get_prediction(x_v,mu1,sigma1,mu2,sigma2,features,samples,y_v)
precision

(27940, 48) (27940, 48)
[[-2.18886782 -0.31223998 -0.88368219 ...  0.61870714  0.63077574
   0.61191785]
 [-2.18886782 -0.30716892 -0.84401342 ...  0.01973181  0.020036
   0.01987001]
 [-2.18886782 -0.30716892 -0.84401342 ... -0.01239973 -0.01220737
  -0.01238624]
 ...
 [ 0.09743488 -0.26491071 -0.39904913 ... -0.01050167 -0.01080887
  -0.01080676]
 [ 0.75084546 -0.23797973 -0.85527783 ... -0.01293859 -0.01262878
  -0.01334894]
 [-0.82954904 -0.30716892 -0.81618606 ... -0.01124189 -0.01095955
  -0.01135704]]
este es samples1 27940
1
esta es la gausiana [8.59820929e-24 2.23353285e-23 3.83186335e-24 6.84461873e-24
 4.57453769e-24 6.92356506e-24 1.18715519e-24 1.55721063e-23
 1.35123745e-27 1.02195373e-24] [1.75338098e-22 3.14519706e-28 2.58867977e-23 4.02574806e-26
 2.01061327e-22 4.07960167e-23 1.54892641e-25 1.01526005e-23
 2.54297188e-26 1.63650653e-23]
este es el denominador [8.74476303e-24 8.56029234e-24 1.45352934e-24 2.53033364e-24
 1.66061943e-24 2.56627506e-24 4.89874150e-25 6.1

c:\Users\52333\Documents\doctorado\ml-md\proyecto\mixture.py:61: RuntimeWarning: invalid value encountered in double_scalars
  gama[i]=p*gausiana[i]/denominador[i]


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan]
2


c:\Users\52333\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\linalg\linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


esta es la gausiana [nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
este es el denominador [nan nan nan nan nan nan nan nan nan nan]
estos son los nk nan nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan]
3
esta es la gausian

KeyboardInterrupt: 